In [2]:
spark

res1: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@2339082b


2부(4~11장)에서는 스파크의 구조적 API를 자세히 살펴보겠음<br/>
구조적 API는 비정형 로그 파일부터 반정형 CSV 파일, 매우 정형적인 Parquet 파일까지 다양한 유형의 데이터를 처리할 수 있음<br/>
구조적 API에는 다음과 같은 3가지 분산 컬렉션 API가 있음<br/>
1. Dataset
2. DataFrame
3. SQL 테이블과 뷰

배치와 스트리밍 처리에서 구조적 API를 사용할 수 있음<br/>
구조적 PAI를 활용하면 배치 작업을 손쉽게 스트리밍 작업으로 변환 가능(반대의 경우에도 마찬가지)<br/>
2부(4~11장)에서는 구조적 API의 내용에 집중하고 스트림 처리 관련 내용은 5부(스트리밍; 20~23장)에서 자세히 알아볼 것임<br/>

구조적 API는 *데이터 흐름을 정의하는 기본 추상화 개념*임<br/>
이 장에서는 반드시 이해해야 하는 3가지 기본 개념을 설명함<br/>

* 타입형(typed)/비타입형(untyped) API의 개념과 차이점
* 핵심 용어
* 스파크가 구조적 API의 데이터 흐름을 해석하고 클러스터에서 실행하는 방식

이어서 특정 데이터 타입이나 데이터 소스를 활용하는 구체적 사례를 알아볼 것임<br/>

4장을 시작하기 전에 스파크의 기본 개념과 정의를 다시 한 번 떠올려보겠음<br/>
스파크는 *transformation의 처리 과정을 정의하는 분산 프로그래밍 모델*임<br/>
사용자가 정의한 다수의 transformation은 DAG로 표현되는 명령을 만들어냄<br/>
action은 하나의 job을 클러스터에서 실행하기 위해 stage와 task로 나누고 DAG 처리 프로세스를 실행함<br/>
transformation과 action으로 다루는 논리적 구조가 바로 DataFrame과 Dataset임<br/>
새로운 DataFrame이나 Dataset을 만들려면 transformaiton을 호출해야 함<br/>
그리고 연산을 시작하거나 사용한 언어에 맞는 데이터 타입으로 변환하려면 action을 호출해야 함<br/>

# 4.1 DataFrame과 Dataset
스파크는 DataFrame과 Dataset이라는 2가지 구조화된 컬렉션 개념을 가지고 있음<br/>
둘 사이의 의미적인 차이점을 알아보기 전에 무엇을 나타내는지 먼저 정의해보겠음<br/>

DataFrame과 Dataset은 *잘 정의된 row와 column을 가지는 분산 테이블 형태의 컬렉션*임<br/>
각 column은 다른 column과 동일한 수의 row를 가져야 함 ('값 없음'은 null로 표시함)<br/>
그리고 컬렉션의 (column별로) 모든 row는 같은 데이터 타입 정보를 가지고 있어야 함<br/>
DataFrame과 Dataset은 *결과를 생성하기 위해 어떤 데이터에 어떤 연산을 적용해야 하는지 정의하는 지연 연산의 실행 계획*이며, *불변성*을 가짐<br/>
DataFrame에 액션을 호출하면 스파크는 transformaiton을 실제로 실행하고 결과를 반환함<br/>
이 과정은 사용자가 원하는 결과를 얻기 위해 row와 column을 처리하는 방법에 대한 계획을 나타냄<br/>

DataFrame과 Dataset을 더 구체적으로 정의하려면 '스키마'를 알아야 함<br/>
스키마는 *분산 컬렉션에 저장할 데이터 타입을 정의하는 방법*임<br/>

# 4.2 스키마
스키마는 DataFrame의 컬럼명과 데이터 타입을 정의함<br/>
스키마는 데이터 소스에서 얻거나 (schema-on-read라고 함) 직접 정의할 수 있음<br/>
스키마는 여러 데이터 타입으로 구성되므로 어떤 데이터 타입이 어느 위치에 있는지 정의하는 방법이 필요함<br/>

# 4.3 스파크의 구조적 데이터 타입 개요
스파크는 사실상 프로그래밍 언어임<br/>
스파크는 실행 계획 수립과 처리에 사용하는 자체 데이터 타입 정보를 가지고 있는 Catalyst 엔진을 사용함<br/>
Catalyst 엔진은 다양한 실행 최적화 기능을 제공함<br/>
스파크는 자체 데이터 타입을 지원하는 여러 언어 API와 직접 매핑되며, 각 언어에 대한 매핑 테이블을 가지고 있음<br/>
파이썬이나 R을 이용해 스파크의 구조적 API를 사용하더라도 대부분의 연산은 파이썬이나 R의 데이터 타입이 아니니 스파크의 데이터 타입을 사용함<br/>
다음 코드 예제는 스칼라나 파이썬이 아닌 스파크의 덧셈 연산을 수행함<br/>

In [3]:
val df = spark.range(500).toDF("number")
df.select(df.col("number") + 10)

df: org.apache.spark.sql.DataFrame = [number: bigint]
res2: org.apache.spark.sql.DataFrame = [(number + 10): bigint]


스파크에서 덧셈 연산이 수행되는 이유는 스파크가 지원하는 언어로 작성된 표현식을 Catalyst 엔진에서 스파크의 데이터 타입으로 변환해 명령을 처리하기 때문임 <br/>
이런 동작이 가능한 이유를 설명하기 전에 Dataset을 먼저 알아보겠음<br/>

## 4.3.1 DataFrame과 Dataset 비교
본질적으로 구조적 API에는 '비타입형'인 DataFrame과 '타입형'인 Dataset이 있음<br/>
물론 DataFrame에도 데이터 타입이 있지만, 스키마에 명시된 데이터 타입의 일치 여부를 **런타임**이 되어서야 확인함<br/>
반면 Dataset은 스키마에 명시된 데이터 타입의 일치 여부를 **컴파일 타임**에 확인함<br/>
Dataset은 JVM 기반의 언어인 스칼라와 자바에서만 지원함<br/>
Dataset의 데이터 타입을 정의하려면 스칼라의 case class나 JavaBean을 사용해야 함<br/>

이 책의 예제는 대부분 DataFrame을 사용함<br/>
스파크의 DataFrame은 Row 타입으로 구성된 Dataset임<br/>
Row 타입을 *스파크가 사용하는 '연산에 최적화된 in-memory 포맷'의 내부적인 표현 방식*임<br/>
Row 타입을 사용하면 garbage collection과 객체 초기화 부하가 있는 JVM 데이터 타입을 사용하는 대신 자체 데이터 포맷을 사용하므로 효율적인 연산이 가능함<br/>
파이썬이나 R에서는 스파크의 Dataset을 사용할 수 없음<br/>
하지만 최적화된 포맷인 DataFrame으로 처리할 수 있음<br/>

DataFrame, 스파크의 데이터 타입, 그리고 스키마를 이해하려면 시간이 필요함<br/>
지금 기억해야 할 것은 **DataFrame을 사용하면 스파크의 최적화된 내부 포맷을 사용할 수 있다는 사실**임<br/>
스파크의 최적화된 내부 포맷을 사용하면 스파크가 지원하는 어떤 언어 API를 사용하더라도 동일한 효과와 효율성을 얻을 수 있음<br/>
만약 컴파일 타임에 엄격한 데이터 타입 검증이 필요하다면 11장 'Dataset'을 참고할 것<br/>

## 4.3.2 컬럼
컬럼은 정수형이나 문자열 같은 **단순 데이터 타입**, 배열이나 맵 같은 **복합 데이터 타입** 그리고 **null 값**을 표현함<br/>
스파크는 데이터 타입의 모든 정보를 추적하며 다양한 컬럼 변환 방법을 제공함<br/>
스파크의 컬럼은 테이블의 컬럼으로 생각할 수 있음<br/>
5장(스트리밍; 20~23장)에서 컬럼에 대해 자세히 알아보겠음<br/>

## 4.3.3 로우
로우는 데이터 레코드임<br/>
DataFrame의 레코드는 Row 타입으로 구성됨<br/>
로우는 SQL, RDD, 데이터 소스에서 얻거나 직접 만들 수 있음<br/>
다음은 range 메서드를 사용해 DataFrame을 생성하는 예제임

In [4]:
spark.range(2).toDF().collect()

res3: Array[org.apache.spark.sql.Row] = Array([0], [1])


위 코드는 Row 객체로 이루어진 배열을 반환함<br/>
*collect()는 전체 RDD 데이터를 가져오는 함수로, 로컬에서 데이터를 처리하고 싶은 때 유용함*<br/>
*단, 전체 RDD 데이터가 단일 컴퓨터의 메모리에 올라올 수 있을 정도의 크기여야 하고 데이터 크기가 너무 크면 사용 불가함*

## 4.3.4 스파크 데이터 타입
스파크는 여러 가지 내부 데이터 타입을 가지고 있음<br/>
특정 데이터 타입의 컬럼을 초기화하고 정의하는 방법을 알아보겠음<br/>
스파크 데이터 타입을 스칼라에서 사용하려면 다음과 같은 코드를 사용함<br/>

In [5]:
import org.apache.spark.sql.types._

val b = ByteType

import org.apache.spark.sql.types._
b: org.apache.spark.sql.types.ByteType.type = ByteType


고정형 DataFrame을 그대로 사용하는 경우는 거의 없으며, 대부분 DataFrame의 처리와 변환을 수행함<br/>
따라서 구조적 API의 실행 과정을 알아야 함<br/>

# 4.4 구조적 API의 실행 과정
이 절에서는 스파크 코드가 클러스터에서 실제 처리되는 과정을 설명함<br/>
이제 구조적 API 쿼리가 사용자 코드에서 실제 실행 코드로 변환되는 과정을 알아볼 것임<br/>
진행 과정은 다음과 같음<br/>
1. DataFrame/Dataset/SQL을 이용해 코드를 작성함
2. 정상적인 코드라면 스파크가 **논리적 실행 계획**으로 변환함
3. 스파크는 **논리적 실행 계획**을 **물리적 실행 계획**으로 변환하며 그 과정에서 추가적인 최적화를 할 수 있는지 확인함
4. 스파크는 클러스터에서 **물리적 실행 계획**(RDD 처리)을 실행함

먼저 실행할 코드를 작성해야 함<br/>
작성한 스파크 코드는 콘솔이나 spark-submit 셸 스크립트로 실행함<br/>
Catalyst Optimizer는 코드를 넘겨받고 실제 실행 계획을 생성함<br/>
마지막으로 스파크는 코드를 실행한 후 결과를 반환함<br/>

## 4.4.1 논리적 실행 계획
첫 번째 실행 단계에서는 사용자 코드를 논리적 실행 계획으로 변환함<br/>

논리적 실행 계획 단계에서는 추상적 transformation만 표현함<br/>
이 단계에서는 driver나 executor의 정보를 고려하지 않음<br/>
그리고 사용자의 다양한 표현식을 최적화된 버전으로 변환함<br/>
이 과정으로 사용자 코드는 **검증 전 논리적 실행 계획(unresolved logical plan)**으로 변환됨<br/>
코드의 유효성과 테이블이나 컬럼의 존재 여부만을 판단하는 과정이므로 아직 실행 계획을 검증하지 않은 상태임<br/>

스파크 분석기(analyzer)는 컬럼과 테이블을 **검증**하기 위해 **카탈로그**, 모든 테이블의 저장소 그리고 DataFrame 정보를 활용함<br/>
필요한 테이블이나 컬럼이 카탈로그에 없다면 검증 전 논리적 실행 계획이 만들어지지 않음<br/>
테이블과 컬럼에 대한 검증 결과는 Catalyst Optimizer로 전달됨<br/>
Catalyst Optimizer는 조건절 푸시 다운(predicate pushing down)이나 선택절 구문을 이용해 논리적 실행 계획을 최적화하는 규칙의 모음임<br/>
필요한 경우 도메인에 최적화된 규칙을 적용할 수 있는 Catalyst Optimizer의 확장형 패키지를 만들 수도 있음<br/>


## 4.4.2 물리적 실행 계획
물리적 실행 계획을 생성하는 과정이 시작됨<br/>
스파크 실행 계획이라고도 불리는 **물리적 실행 계획**은 논리적 실행 계획을 클러스터 환경에서 실행하는 방법을 정의함<br/>
다양한 물리적 실행 전략을 생성하고 비용 모델을 이용해서 비교한 후 최적의 전략을 선택함<br/>
비용을 비교하는 한 가지 예는 사용하려는 테이블의 크기나 파티션 수 등의 물리적 속성을 고려해 지정된 join 연산 수행에 필요한 비용을 계산하고 비교하는 것임<br/>

물리적 실행 계획은 일련의 RDD와 transformation으로 변환됨<br/>
스파크는 DataFrame, Dataset, SQL로 정의된 쿼리를 RDD transformation으로 컴파일함<br/>
따라서 스파크를 '컴파일러'라고 부르기도 함<br/>

## 4.4.3 실행
스파크는 물리적 실행 계획을 선정핞 다음 저수준 프로그래밍 인터페이스인 RDD를 대상으로 모든 코드를 실행함<br/>
스파크는 런타임에 전체 task나 stage를 제거할 수 있는 자바 파이트 코드를 생성해 추가적인 최적화를 수행함<br/>
마지막으로 스파크는 처리 결과를 사용자에게 반환함<br/>

# 4.5 정리
이 장에서는 스파크의 구조적 API를 알아보았음<br/>
그리고 사용자 코드가 물리적인 실행 코드로 변환되는 과정도 함께 알아보았음<br/>
다음 장에서는 구조적 API 핵심 개념과 주요 기능의 사용법을 자세히 알아보겠음<br/>